In [ ]:
import cv2
import numpy as np
import random

def remove_overlapping_circles(circles, distance_threshold=15):
    """
    Removes overlapping circles by ensuring that the distance between
    any two circle centers is greater than `distance_threshold`.
    If two circles overlap, we keep the one with the larger radius.
    """
    if len(circles) == 0:
        return circles
    
    final_circles = []
    for c in circles:
        x1, y1, r1 = c
        keep = True
        for fc in final_circles:
            x2, y2, r2 = fc
            dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            if dist < distance_threshold:
                # They overlap; keep the circle with the larger radius
                if r1 > r2:
                    # Replace the old circle
                    final_circles.remove(fc)
                    final_circles.append(c)
                keep = False
                break
        if keep:
            final_circles.append(c)
    
    return final_circles

def detect_coins(image_path, param1, param2, minRadius, maxRadius,
                 distance_threshold=15, show=False):
    """
    Detects coins using HoughCircles with given hyperparameters,
    removes overlapping circles, and optionally displays the result.
    Returns the number of coins detected.
    """
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read image from:", image_path)
        return 0
    
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # Hough Circle detection
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=30,      # You can adjust this based on coin spacing
        param1=param1,   # Canny high threshold
        param2=param2,   # Accumulator threshold for center detection
        minRadius=minRadius,
        maxRadius=maxRadius
    )
    
    # If no circles found, return 0
    if circles is None or len(circles) == 0:
        if show:
            print("No circles detected with these parameters.")
        return 0
    
    # Convert circles to integer format
    circles = np.round(circles[0, :]).astype("int")
    
    # Remove overlapping circles
    circles = remove_overlapping_circles(circles, distance_threshold=distance_threshold)
    
    # Draw circles in random colors if show=True
    if show:
        for (x, y, r) in circles:
            color = (
                random.randint(0, 255),
                random.randint(0, 255),
                random.randint(0, 255)
            )
            cv2.circle(output, (x, y), r, color, 2)  # Circle boundary
            cv2.circle(output, (x, y), 2, (0, 0, 255), 3)  # Center point
        
        # Show the result
        cv2.imshow(f"Hough Circles (p1={param1}, p2={param2}, r=[{minRadius},{maxRadius}])", output)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    
    return len(circles)


if __name__ == "__main__":
    image_path = "c1.jpg"  # Replace with your image path
    num_coins = detect_coins(image_path, param1=50, param2=30, minRadius=60, maxRadius=100, distance_threshold=18, show=True)
    print("Number of coins detected:", num_coins)

In [ ]:
import cv2
import numpy as np
import random

def remove_overlapping_circles(circles, distance_threshold=15):
    """
    Removes overlapping circles by ensuring that the distance between
    any two circle centers is greater than `distance_threshold`.
    If two circles overlap, we keep the one with the larger radius.
    """
    if len(circles) == 0:
        return circles
    
    final_circles = []
    for c in circles:
        x1, y1, r1 = c
        keep = True
        for fc in final_circles:
            x2, y2, r2 = fc
            dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            if dist < distance_threshold:
                # They overlap; keep the circle with the larger radius
                if r1 > r2:
                    # Replace the old circle
                    final_circles.remove(fc)
                    final_circles.append(c)
                keep = False
                break
        if keep:
            final_circles.append(c)
    
    return final_circles

def detect_coins(image_path, param1, param2, minRadius, maxRadius,
                 distance_threshold=15, show=False):
    """
    Detects coins using HoughCircles with given hyperparameters,
    removes overlapping circles, and optionally displays the result.
    Uses explicit edge detection (Canny) for coin detection.
    Returns the number of coins detected.
    """
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read image from:", image_path)
        return 0
    
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # Explicit edge detection using Canny
    # Adjust the thresholds as needed; here we use param1 to derive the low threshold
    edges = cv2.Canny(blurred, param1 / 2, param1)
    
    # Hough Circle detection using the edge-detected image
    circles = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=30,      # Adjust based on coin spacing
        param1=param1,   # High threshold for Canny (again, used internally by HoughCircles)
        param2=param2,   # Accumulator threshold for center detection
        minRadius=minRadius,
        maxRadius=maxRadius
    )
    
    # If no circles are found, return 0
    if circles is None or len(circles) == 0:
        if show:
            print("No circles detected with these parameters.")
        return 0
    
    # Convert circles to integer format
    circles = np.round(circles[0, :]).astype("int")
    
    # Remove overlapping circles
    circles = remove_overlapping_circles(circles, distance_threshold=distance_threshold)
    
    # Draw circles in random colors if show=True
    if show:
        for (x, y, r) in circles:
            color = (
                random.randint(0, 255),
                random.randint(0, 255),
                random.randint(0, 255)
            )
            cv2.circle(output, (x, y), r, color, 2)  # Circle boundary
            cv2.circle(output, (x, y), 2, (0, 0, 255), 3)  # Center point
        
        # Show the images
        #cv2.imshow("Edge Detected Image", edges)
        window_name = f"Hough Circles (p1={param1}, p2={param2}, r=[{minRadius},{maxRadius}])"
        #cv2.imshow(window_name, output)
        
        # Poll for key press or window close
        while True:
            key = cv2.waitKey(1)  # wait 1ms for a key event
            # Check if either window has been closed
            if (cv2.getWindowProperty("Edge Detected Image", cv2.WND_PROP_VISIBLE) < 1 or 
                cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1):
                break
    
            if key != -1:
                break
        
        cv2.destroyAllWindows()
    
    return len(circles)


if __name__ == "__main__":
    image_path = "c1.jpg"  # Replace with your image path
    num_coins = detect_coins(image_path, param1=50, param2=30, minRadius=25, maxRadius=40, distance_threshold=12, show=True)
    print("Number of coins detected:", num_coins)


Number of coins detected: 120


In [4]:
import cv2
import numpy as np
import random

def remove_overlapping_circles(circles, distance_threshold=15):
    """
    Removes overlapping circles by ensuring that the distance between
    any two circle centers is greater than `distance_threshold`.
    If two circles overlap, we keep the one with the larger radius.
    """
    if len(circles) == 0:
        return circles
    
    final_circles = []
    for c in circles:
        x1, y1, r1 = c
        keep = True
        for fc in final_circles:
            x2, y2, r2 = fc
            dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            if dist < distance_threshold:
                # They overlap; keep the circle with the larger radius
                if r1 > r2:
                    # Replace the old circle
                    final_circles.remove(fc)
                    final_circles.append(c)
                keep = False
                break
        if keep:
            final_circles.append(c)
    
    return final_circles

def detect_coins(image_path, param1, param2, minRadius, maxRadius,
                 distance_threshold=15, show=False):
    """
    Detects coins using HoughCircles with given hyperparameters,
    removes overlapping circles, and optionally displays the result.
    Uses explicit edge detection (Canny) for coin detection.
    Returns the number of coins detected.
    """
    # Read image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read image from:", image_path)
        return 0
    
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (21, 21), 4)
    #cv2.imshow("Edge Detected Image",  blurred)
    #cv2.waitKey(1)
    #cv2.destroyAllWindows()
    
    
    # Explicit edge detection using Canny
    # Adjust the thresholds as needed; here we use param1 to derive the low threshold
    edges = cv2.Canny(blurred, param1 / 2, param1)
    
    # Hough Circle detection using the edge-detected image
    circles = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=30,      # Adjust based on coin spacing
        param1=param1,   # High threshold for Canny (again, used internally by HoughCircles)
        param2=param2,   # Accumulator threshold for center detection
        minRadius=minRadius,
        maxRadius=maxRadius
    )
    
    # If no circles are found, return 0
    if circles is None or len(circles) == 0:
        if show:
            print("No circles detected with these parameters.")
        return 0
    
    # Convert circles to integer format
    circles = np.round(circles[0, :]).astype("int")
    
    # Remove overlapping circles
    circles = remove_overlapping_circles(circles, distance_threshold=distance_threshold)
    
    # Draw circles in random colors if show=True
    if show:
        for (x, y, r) in circles:
            color = (
                random.randint(0, 255),
                random.randint(0, 255),
                random.randint(0, 255)
            )
            cv2.circle(output, (x, y), r, color, 2)  # Circle boundary
            cv2.circle(output, (x, y), 2, (0, 0, 255), 3)  # Center point
        
        # Optionally show the edge-detected image for reference
        cv2.imshow("Edge Detected Image", edges)
        cv2.imshow(f"Hough Circles (p1={param1}, p2={param2}, r=[{minRadius},{maxRadius}])", output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return len(circles)

# Example usage:b
if __name__ == "__main__":
    image_path = "c2.jpeg"  # Replace with your image path
    num_coins = detect_coins(image_path, 50,30,9,40,20, show=True)
    print("Number of coins detected:", num_coins)


Number of coins detected: 1


In [ ]:
import cv2
import numpy as np

def detect_and_count_coins(image_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return
    original = image.copy()
    
    # Preprocessing: Convert to grayscale and blur
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # Edge detection
    edges = cv2.Canny(gray, 50, 150)
    edges = cv2.dilate(edges, None, iterations=2)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Filter contours by circularity and area
    min_area = 500  # Adjust based on your image
    coins = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area < min_area:
            continue
        perimeter = cv2.arcLength(cnt, True)
        if perimeter == 0:
            continue
        circularity = (4 * np.pi * area) / (perimeter ** 2)
        if circularity > 0.7:
            coins.append(cnt)
    
    # Part a: Visualize detected coins
    detected_image = original.copy()
    cv2.drawContours(detected_image, coins, -1, (0, 255, 0), 2)
    cv2.imshow('Detected Coins', detected_image)
    cv2.waitKey(0)
    
    # Part b: Segment each coin
    for idx, cnt in enumerate(coins):
        mask = np.zeros_like(gray)
        cv2.drawContours(mask, [cnt], -1, 255, -1)
        segmented = cv2.bitwise_and(original, original, mask=mask)
        x, y, w, h = cv2.boundingRect(cnt)
        cropped = segmented[y:y+h, x:x+w]
        cv2.imshow(f'Segmented Coin {idx + 1}', cropped)
        cv2.waitKey(0)
    
    cv2.destroyAllWindows()
    
    # Part c: Count coins
    count = len(coins)
    print(f'Total number of coins: {count}')
    return count

# Example usage
detect_and_count_coins('coins.jpg')

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/hlgsagar/.local/lib/python3.11/site-packages/cv2/qt/plugins"


In [ ]:
import cv2
import numpy as np

def detect_segment_and_count_coins(image_path):
    
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Image not found.")
        return 0
    
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
 
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    

    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50,
                               param1=50, param2=30, minRadius=10, maxRadius=100)
    
    if circles is None:
        print("No coins detected.")
        return 0
    
    
    if circles.ndim == 3:
        circles = np.round(circles[0, :]).astype("int")
    else:
        circles = np.round(circles).astype("int")
    
    total_coins = len(circles)
    segmented_coins = []
    
    for (x, y, r) in circles:
       
        cv2.circle(output, (x, y), r, (0, 255, 0), 2)
        cv2.circle(output, (x, y), 2, (0, 0, 255), 3)
        
        
        mask = np.zeros_like(gray)
        cv2.circle(mask, (x, y), r, 255, -1)
        segmented = cv2.bitwise_and(image, image, mask=mask)
        
      
        y1, y2 = max(0, y - r), min(image.shape[0], y + r)
        x1, x2 = max(0, x - r), min(image.shape[1], x + r)
        segmented_cropped = segmented[y1:y2, x1:x2]
        
        
        if segmented_cropped.size != 0:
            segmented_coins.append(segmented_cropped)
    
   
    cv2.imshow('Detected Coins', output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    for idx, coin in enumerate(segmented_coins):
        window_name = f'Segmented Coin {idx + 1}'
        cv2.imshow(window_name, coin)
        cv2.waitKey(0)
        cv2.destroyWindow(window_name)
    
    
    print(f"Total number of coins detected: {total_coins}")
    return total_coins


detect_segment_and_count_coins('coins.jpg')


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/hlgsagar/.local/lib/python3.11/site-packages/cv2/qt/plugins"


In [ ]:
import cv2
import numpy as np
import random

def remove_overlapping_circles(circles, distance_threshold=15):
    if len(circles) == 0:
        return circles
    final_circles = []
    for c in circles:
        x1, y1, r1 = c
        keep = True
        for fc in final_circles:
            x2, y2, r2 = fc
            dist = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
            if dist < distance_threshold:
                if r1 > r2:
                    final_circles.remove(fc)
                    final_circles.append(c)
                keep = False
                break
        if keep:
            final_circles.append(c)
    return final_circles

def detect_coins(image_path, param1, param2, minRadius, maxRadius, distance_threshold=15, show=False):
    image = cv2.imread(image_path)
    if image is None:
        print("Error: Could not read image from:", image_path)
        return 0
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    edges = cv2.Canny(blurred, param1 / 2, param1)
    circles = cv2.HoughCircles(
        edges,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=30,
        param1=param1,
        param2=param2,
        minRadius=minRadius,
        maxRadius=maxRadius
    )
    if circles is None or len(circles) == 0:
        if show:
            print("No circles detected with these parameters.")
        return 0
    circles = np.round(circles[0, :]).astype("int")
    circles = remove_overlapping_circles(circles, distance_threshold=distance_threshold)
    if show:
        for (x, y, r) in circles:
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            cv2.circle(output, (x, y), r, color, 2)
            cv2.circle(output, (x, y), 2, (0, 0, 255), 3)
        window_name = f"Hough Circles (p1={param1}, p2={param2}, r=[{minRadius},{maxRadius}])"
        while True:
            key = cv2.waitKey(1)
            if (cv2.getWindowProperty("Edge Detected Image", cv2.WND_PROP_VISIBLE) < 1 or 
                cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1):
                break
            if key != -1:
                break
        cv2.destroyAllWindows()
    return len(circles)

if __name__ == "__main__":
    image_path = "c1.jpg"
    num_coins = detect_coins(image_path, param1=50, param2=30, minRadius=25, maxRadius=40, distance_threshold=12, show=True)
    print("Number of coins detected:", num_coins)


In [4]:
import pandas as pd
data = pd.read_csv('res.csv')

In [5]:
data.head()

,param1,param2,minRadius,maxRadius,distance_threshold,coin_count
0,50,20,10,40,12,356
1,50,20,10,40,13,356
2,50,20,10,40,14,356
3,50,20,10,40,15,356
4,50,20,10,40,16,356


In [10]:
new_data =  data[ (data['coin_count']>16) &  (data['coin_count']<20) ]
new_data.shape

(81, 6)

In [1]:
data3=data2[data2['coin_count']==18]


NameError: name 'data2' is not defined